In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as pyo
import seaborn as sns
from natsort import index_natsorted
from scipy.stats import norm
from __future__ import print_function
import ipywidgets as widgets
from ipywidgets import fixed, interact, interact_manual, interactive
%load_ext line_profiler
%matplotlib inline
pyo.init_notebook_mode(connected=True)

In [2]:
sulIntro=1936

In [3]:
df = pd.read_csv("./all_rgiout_2022-04-12", sep="\t")
df = df.sort_values(
    by=["Accession_Number"],
    ascending=True,
    key=lambda x: np.argsort(index_natsorted(df["Accession_Number"])),
    ignore_index=True,
)
df[["Species", "Subspecies"]] = df["Strain"].str.split("subsp.", 1, expand=True)
df = df.replace(np.nan, "Null", regex=True)

rgiintegron = pd.read_csv('./rgi_subset_integronoverlap', sep='\t', names=df.columns.values)
rgiintegron["Species"] = rgiintegron["Strain"].str.split("subsp.", 1, expand=True)
rgiintegron["Subspecies"] = "Null"
contigdf = pd.concat([df['Accession_Number'], df['Contig'].str.split('_', expand=True)[0].str.split('|').str[-1]], axis=1).copy()

ecoli_mlplasmid_out = pd.read_csv('../3_analysis-plasmid/merged-Escherichia_coli-mlplasmidout', sep='\t')
ecoli_mlplasmid_out['Contig'] = ecoli_mlplasmid_out['Contig_name'].str.split(' ', expand=True)[0]
ecoliplasmids_df = contigdf.loc[contigdf['Accession_Number'].isin(ecoli_mlplasmid_out['AccNum'])]
ecoliplasmids_df = ecoliplasmids_df.loc[ecoliplasmids_df[0].isin(ecoli_mlplasmid_out['Contig'])]
ecoliplasmids_df = df.loc[ecoliplasmids_df.index].copy()

FileNotFoundError: [Errno 2] No such file or directory: './all_rgiout_2022-04-12'

In [ ]:
sul_resist = df.loc[df['Drug Class'].str.contains('sulfonamide')]
sul_resist = df.loc[df['Best_Hit_ARO'].str.contains('sul')]

In [ ]:
px.bar(
    sul_resist,
    x="Year_Cultured",
    y=sul_resist['Year_Cultured'].map((1/sul_resist['Year_Cultured'].value_counts()).to_dict()),
    hover_data=["Accession_Number", "Resistance Mechanism", "Best_Hit_ARO", "Drug Class", "AMR Gene Family", "Strain"],
    color="Best_Hit_ARO"
)

In [ ]:
import matplotlib.pyplot as plt
from dna_features_viewer import BiopythonTranslator
from Bio import SeqIO
import numpy as np

record_dict = SeqIO.to_dict(SeqIO.parse("/n/scratch3/users/a/ak586/microtrawler/1_sequences/NCTC/ena/NCTC9127/GCA_900452485/PROKKA_04122022/PROKKA_04122022.gbk", "genbank"))
# sub_record = record_dict['UGNE01000001.1'][1405899:1417399]
sub_record = record_dict['UGNE01000001.1'][1406000:1416000]
# print(sub_record)
for features in sub_record.features:
    if 'gene' in features.qualifiers:
        gname = features.qualifiers['gene'][0]
        # print(features.qualifiers)
        if gname == 'bin3':
            features.qualifiers['gene'] = ['Putative Tn552 DNA-invertase']
        if gname == 'glmM_2':
            features.qualifiers['gene'] = ['glmM']
        if gname == 'folP_2':
            features.qualifiers['gene'] = ['sul2']
        if gname == 'chuR_2':
            features.qualifiers['gene'] = ['chuR']
    else:
        prodName = features.qualifiers['product'][0]
        if 'NCBIFAM' in prodName:
            NewprodName = prodName.split(': ')[1]
            features.qualifiers['product'] = [NewprodName]
            if 'metalloregulator' in NewprodName:
                features.qualifiers['product'] = ['metalloregulator ArsR/SmtB TF']
                # print(features.qualifiers)
        # print(features.qualifiers)
graphic_record = BiopythonTranslator().translate_record(sub_record)
ax, _ = graphic_record.plot(figure_width=15, with_ruler=False, strand_in_label_threshold=7)
ax.figure.savefig('sul2-neigh.png', bbox_inches='tight', dpi=200)

In [ ]:
import alv
from Bio import AlignIO
msa = AlignIO.read('/n/scratch3/users/a/ak586/microtrawler/1_sequences/NCTC/ena/NCTC9127/GCA_900452485/hey.fa', 'fasta')
alv.view(msa, width=160, dotted=True)


In [ ]:
alv.view?

In [ ]:
px.histogram(
    df.loc[df['Accession_Number'].isin(ecoli_mlplasmid_out['AccNum'])].drop_duplicates(subset=['Accession_Number']),
    x='Year_Cultured'
)

In [ ]:
px.bar(
    rgiintegron,
    x="Year_Cultured",
    y=rgiintegron['Year_Cultured'].map((1/rgiintegron['Year_Cultured'].value_counts()).to_dict()),
    hover_data=["Accession_Number", "Resistance Mechanism", "Best_Hit_ARO", "Drug Class", "AMR Gene Family", "Strain"],
    color="Best_Hit_ARO"
)

In [ ]:
sul_resist['Strain'].value_counts()

In [ ]:
df.drop_duplicates(subset=['Accession_Number'])['Strain'].value_counts()['Klebsiella pneumoniae']

In [ ]:
rgiintegron